In [1]:
from collections import Counter
import pandas as pd
import json
import os

In [2]:
root = "."
pathogen_code = 'mtuberculosis'

In [3]:
# Load cleaned assays
ASSAYS_CLEANED = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, "assays_cleaned.csv"))

# Define PATH to parameters
PATH_TO_PARAMETERS = os.path.join(root, "..", "output", pathogen_code, 'parameters')

TARGET_TYPE_CURATED = []

# Iterating over assays
for assay_id, activity_type, unit in ASSAYS_CLEANED[['assay_id', 'activity_type', 'unit']].values:

    # Prepare filename
    filename = "_".join([str(assay_id), str(activity_type), str(unit), 'parameters']) + ".json"
    
    # Read JSON file
    with open(os.path.join(PATH_TO_PARAMETERS, filename), "r") as file:
        par = json.load(file)

    # Store results
    TARGET_TYPE_CURATED.append(par['target_type_curated'])

# Complete table
ASSAYS_CLEANED['target_type_curated'] = TARGET_TYPE_CURATED


In [4]:
Counter(ASSAYS_CLEANED['target_type'])

Counter({'ORGANISM': 8664,
         'SINGLE PROTEIN': 954,
         'UNCHECKED': 786,
         'PROTEIN COMPLEX': 84,
         'SUBCELLULAR': 17,
         'NON-MOLECULAR': 13,
         'ADMET': 2,
         'PROTEIN FAMILY': 1})

In [5]:
len(ASSAYS_CLEANED[ASSAYS_CLEANED['target_type'] == 'UNCHECKED'])

786

In [6]:
len(ASSAYS_CLEANED[(ASSAYS_CLEANED['target_type'] == 'UNCHECKED') & (ASSAYS_CLEANED['target_type_curated'] == 'UNCHECKED')])

5

In [11]:
ASSAYS_CLEANED[(ASSAYS_CLEANED['target_type'] == 'UNCHECKED') & (ASSAYS_CLEANED['target_type_curated'] == 'CELL-LINE')]

,assay_id,assay_type,assay_organism,doc_chembl_id,target_type,target_chembl_id,target_organism,activity_type,unit,canonical_unit,activities,nan_values,cpds,direction,activity_comment_counts,standard_text_count,target_type_curated
